# **A/B Testing**

## **What is A/B Testing?**
The purpose of A/B testing is to compare different content and design samples in pairs. However, this purpose may differ depending on the system in which we perform the test. In addition to being used mostly on applications, websites and advertisements, it is a test that can be applied in almost all decisions that can be taken thanks to the digital competencies we have. AB Testing outputs help us make accurate predictions. In summary, it can be said that they are experiments in which only one of more than one variable is changed and which variant performs better under equal conditions.

## **How to Do A/B Testing?**
First of all, a decision should be made about which variable A B test will be performed on. To go through a common example in digital marketing, let's talk about an experiment to be done on the CTA (Call-to-Action) text of an ad image. Let's assume that the current advertisement image says "Make an Appointment Now", and if we are wondering how it will perform, "Call Now". In the final stage, these two alternative texts should be evaluated under equal conditions.

When doing A/B testing, there are some factors that you need to pay attention to in order to be able to do it in the most effective way.

**Period**: Both versions should be tested at the same time. If version A is tested in September and version B is tested in November, we won't know if the results are different because of timing and not because of different elements.

**Randomness**: Test groups should be randomly and equally divided so that the results are unbiased.

**Time**: If the test is only done for a short time, the results may not be enough to make a decision.

The correct test method must be selected. statistical hypotheses must be properly established and tested.

An example hypothesis to be established for the problem might be as follows:

- H0: There is no statistically significant difference between the conversion rate value of the old system and the conversion rate value of the new system.

- H1: There is a statistically significant difference between the conversion rate value of the old system and the conversion rate value of the new system.

## **Steps for the A/B Test**
Prepare the Data: Ensure your dataset is clean and separated by old_page and new_page.

**Choose a Metric**: 

Decide which metric (Clicks, Purchases, Earnings, etc.) you will test.

**Set up Hypotheses**: 

Based on the metric you choose:

- H0: There is no difference in [Clicks/Purchases/Earnings] between the old_page and the new_page.

- H1: There is a significant difference in [Clicks/Purchases/Earnings] between the old_page and the new_page.

**Check for Normality**:

Apply the Shapiro-Wilk test (shapiro()) to check if your data follows a normal distribution.

**Test for Variance**:
Use Levene's Test (levene()) to check if the variances between the two groups are equal.

**Select a Statistical Test**:
- **Parametric Test**: If the data is normally distributed and the variances are equal, use a t-test (ttest_ind()).

- **Non-Parametric Test**: If the data does not follow a normal distribution, use a non-parametric test like the Mann-Whitney U test (mannwhitneyu()).

**Interpret the Results:**

Based on the p-value, determine if you can reject the null hypothesis.
- If p-value < 0.05, reject H0 (there is a statistically significant difference).

- If p-value > 0.05, fail to reject H0 (no significant difference).

In [1]:
import numpy as np
import pandas as pd
from scipy.stats import shapiro, levene, ttest_ind, mannwhitneyu
from statsmodels.stats.proportion import proportions_ztest

Let's read the dataset. In order to test, we will need two groups, the Control Group and the Test Group.

**CONTROL GROUP**: We can read the data set of the control group as follows.

In [2]:
file_path = "ab_testing.xlsx"
df_control = pd.read_excel(file_path,sheet_name= "Control Group")
df_control.head()

,Impression,Click,Purchase,Earning
0,82529.459271,6090.077317,665.211255,2311.277143
1,98050.451926,3382.861786,315.084895,1742.806855
2,82696.023549,4167.965750,458.083738,1797.827447
3,109914.400398,4910.882240,487.090773,1696.229178
4,108457.762630,5987.655811,441.034050,1543.720179


For a first impression of our control group dataset, we can take a quick look.

In [3]:
df_control.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


**TEST GROUP**: After the control group, we read the data set of the test group as follows.

In [4]:
df_test = pd.read_excel(file_path,sheet_name= "Test Group")
df_test.head()

,Impression,Click,Purchase,Earning
0,120103.503796,3216.547958,702.160346,1939.611243
1,134775.943363,3635.082422,834.054286,2929.405820
2,107806.620788,3057.143560,422.934258,2526.244877
3,116445.275526,4650.473911,429.033535,2281.428574
4,145082.516838,5201.387724,749.860442,2781.697521


In [5]:
df_test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40 entries, 0 to 39
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Impression  40 non-null     float64
 1   Click       40 non-null     float64
 2   Purchase    40 non-null     float64
 3   Earning     40 non-null     float64
dtypes: float64(4)
memory usage: 1.4 KB


## **Formulating a Hypothesis**
1. Assumption is Normality

2. Assumption is Variance Homogenity

- If assumptions are provided then two independent sample t-tests (parametric testing) will be applied

- If assumptions are not provided than Mannwhitneyu test (non-parametric test) will be applied

Therefore, we should first look at the assumption of normality.

We will perform hypothesis testing. It is necessary to interpret whether the results are statistically significant. Normality assumption check is considered as the first step of this test.

## **Normality Assumption**
It is possible to use various normality tests in order to reveal whether the data are suitable for normal distribution. The most well-known of these tests are Chi-Square, Kolmogorow-Smirnov, Lilliefors and Shapiro-Wilk normality tests. Among these, we will use the SHAPIRO WILK-W test.

### **Shapiro Wilk-W Test**
Shapiro-Wilk and Kolmogorov-Smirnov are the most commonly used tests for the normal distribution test, and their hypotheses are basically the same. In contrast to other comparison tests the Shapiro-Wilk test is only applicable to check for normality.

- **H0**: The variable has a normal distribution. p-value > 0.05

- **H1**: The variable does not have a normal distribution. p-value < 0.05

The null-hypothesis of this test is that the population is normally distributed. Thus, if the p value is less than the chosen alpha level, then the null hypothesis is rejected and there is evidence that the data tested are not normally distributed. On the other hand, if the p value is greater than the chosen alpha level, then the null hypothesis can not be rejected.

- **H0**: Normal distribution assumption is provided.

- **H1**: Normal distribution assumption is not provided.

In [6]:
df_control['Purchase'].mean()

np.float64(550.8940587702316)

In [7]:
df_test['Purchase'].mean()

np.float64(582.1060966484677)

In [8]:
test_stat, pvalue = shapiro(df_control["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

test_stat, pvalue = shapiro(df_test["Purchase"])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 0.9773, p-value = 0.5891
Test Stat = 0.9589, p-value = 0.1541


**H0 cannot be rejected** for each group because *p-value = 0.1541 and 0.5891 > 0.05*, so that is, the normal distribution assumption of is provided.

Now that the variance of normality is satisfied, we should be able to control the homogeneity of the variance.

## **Variance Homogenity Assumption**
### **Levene's Test**
- **H0**: Variances are Homogeneous.

- **H1**: Variances are not Homogeneous.

Levene's test is used to test the homogeneity of group variances. It is run when groups are two or more. In other words, it is used in independent groups t test and Anova test. It is not used in the t test in dependent groups where the number of groups is odd. The sig (p) value found as a result of the Levene test is checked. If this value is greater than 0.05, we say that there is no difference. There is no difference between the groups. So we say the variances are equal (homogeneous). In this case, the values ​​in the first row (t value, p value) are taken into account. If equality of variances is not achieved, the values ​​in the second row (t value, p value) are taken into account. For example: p of the Levene test Let the value be 0.949. Since this value is greater than 0.05, we say that the variances are equal (Equal variances assumed). And in the rest of the first line, we look at the values. Let the p value of the sample t test be found to be 0.123. If this value is less than/equal to p=0.05, the hypothesis is confirmed. Otherwise, it will not be validated. The equation in our example was: 0.123>0.05. In this case, our hypothesis was not confirmed. H0 accepted, H1 rejected.

In [9]:
test_stat, pvalue = levene(df_control['Purchase'],
                           df_test['Purchase'])
print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = 2.6393, p-value = 0.1083


**H0 cannot be rejected** because *p-value = 0.1083 > 0.05*, so that is, homogeneous variance distribution is provided.

We can apply Two Independent Sample T-Tests which is categorized as parametric test, since the assumptions are satisfied.

## **Independent Samples T Test**
The independent sample t-test is used to test whether there is a statistically significant difference between two independent groups by looking at the means. This test is a parametric test and some assumptions (pre-requisites) must be fulfilled in order to report the results of the test.

We checked these necessary conditions in the previous step. Again, the dependent variable in each group should be normally distributed (Normality). Variances should be homogeneous. (Homogeneity)

Apart from these two conditions, the following conditions must also be met:

- The dependent variable must be continuous.

- The argument must be categorical.

- The dataset should not contain outliers.

**Note**: The Independent Samples t Test can only compare the means for two (and only two) groups. It cannot make comparisons among more than two groups. If you wish to compare the means across more than two groups, you will likely want to run an ANOVA.

In [10]:
test_stat, pvalue = ttest_ind(df_control["Purchase"],
                              df_test["Purchase"],
                              equal_var=True)

print('Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))

Test Stat = -0.9416, p-value = 0.3493


**H0 cannot be rejected** because *p-value = 0.3493 > 0.05* . In other words, there is no statistically significant difference between the purchase averages of the control and test groups. There is no significant difference between maximum bidding and average bidding.

## **Mann–Whitney U test**
It is a non-parametric method used to compare the means of two independent groups in a distribution that does not show normal distribution. Since this test is a non-parametric test, no assumptions are necessary regarding the distribution of the scores obtained.

A few necessary prerequisites for this test can be listed as follows:

- The independent variable must be categorical.

- The dependent variable must be continuous or sequential.

- The sample selected from the population should be random.

- The scores obtained from the groups should not show normal distribution.


## **Functionalization of A/B Test**

In [11]:
def hypothesis_checker(df_A, df_B, col_name):
    
    test_stat_A, pvalue_A = shapiro(df_A[col_name])
    test_stat_B, pvalue_B = shapiro(df_B[col_name])
    test_stat_var, pvalue_var = levene(df_A[col_name], df_B[col_name])
    
    print('*Normalization Check:')
    print('Shapiro Test for Control Group, Stat = %.4f, p-value = %.4f' % (test_stat_A, pvalue_A))
    print('Shapiro Test for Test Group, Stat = %.4f, p-value = %.4f' % (test_stat_B, pvalue_B))
    
    print('*Variance Check:')
    print('Levene Test Stat = %.4f, p-value = %.4f' % (test_stat_var, pvalue_var))
    
    if pvalue_A < 0.05 and pvalue_B < 0.05:
        test_stat, pvalue = mannwhitneyu(df_A[col_name], df_B[col_name])

        print(" ")
        print('Mann–Whitney U Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
        print(f'H0 hypothesis {"NOT REJECTED" if pvalue > 0.05 else "REJECTED"} in Mann–Whitney U Test')
        
    else:
        test_stat, pvalue = ttest_ind(df_A[col_name], df_B[col_name], equal_var=(True if pvalue_var > 0.05 else False))

        print(" ")
        print('Independent Samples T Test Stat = %.4f, p-value = %.4f' % (test_stat, pvalue))
        print(f'H0 hypothesis {"NOT REJECTED" if pvalue > 0.05 else "REJECTED"}, Independent Samples T Test')

In [12]:
hypothesis_checker(df_control, df_test, 'Earning')

*Normalization Check:
Shapiro Test for Control Group, Stat = 0.9756, p-value = 0.5306
Shapiro Test for Test Group, Stat = 0.9780, p-value = 0.6163
*Variance Check:
Levene Test Stat = 0.3532, p-value = 0.5540
 
Independent Samples T Test Stat = -9.2545, p-value = 0.0000
H0 hypothesis REJECTED, Independent Samples T Test


The normality is provided beacuse *p-value = 0.5306 and 0.6163 > 0.05* that is, **H0 can not rejected**. The variance homogenity is provided beacuse *p-value = 0.5540 > 0.05* that is, **H0 can not rejected**. Due to the normality and variance homogenity assumptions are provided, we should perform Independent Samples T Test (parametric test).

As a result of Independent Samples T Test, there is a statistically significant difference between them because *p-value = 0.0000 < 0.05* so **H0 is rejected**. Considering the gain in the test group, the test group earns more. That's why we should choose average bidding.

In [13]:
print('control_mean', df_control['Earning'].mean())
print('test_mean', df_test['Earning'].mean())

control_mean 1908.568299802749
test_mean 2514.8907326506173


## **Another Aspect - Conversion of Impression to Click**

### **Two Proportion Z-Test**

The Two-Proportions Z-Test is used to compare two ratios.

Let's assume we have 2 groups of individuals: + Group 1, individuals with skin cancer: n = 100 + Group 2, healthy individuals: n = 100

Number of smokers in each group:

490 individuals from group 1 male: 98/100 = 98%

400 individuals from group 2 male: 80/100 = 80% In total:

Percentage of male: 89%

Percentage of female: 11%

What we want to investigate is whether the proportion of gender in the two groups is the same.

H0: μ1 = μ2 (the two population proportions are equal) The alternative hypothesis can be either two-tailed, left-tailed, or right-tailed

**H0 : There is a statistically significant difference between the click rate averages of control and test groups.**

**H1 : There is no statistically significant difference between the click rate averages of control and test groups.**

In [14]:
click_a = df_control["Click"].sum()
impression_a = df_control["Impression"].sum()
print(click_a/impression_a)

0.05014831092596444


In [15]:
click_b = df_test["Click"].sum()
impression_b = df_test["Impression"].sum()
print(click_b/impression_b)

0.032922333085396625


In [16]:
nclicks = np.array([click_a,click_b])

nad_views = np.array([impression_a,impression_b])

In [17]:
proportions_ztest(count=nclicks,nobs=nad_views)

(np.float64(129.3304801715903), np.float64(0.0))

There is a statistically significant difference between them

because *p-value = 0.0000 < 0.05* so **H0 is rejected**.

Based on the current average click-through rates,

- the maximum bidding 5 out of 100 people click on the ad, while

- the ad the average bidding 3 out of 100 people click on.

So, if the company wants to increase the click-through rate, it may prefer maximum bidding.

## **Conclusion**
Running an A/B test that directly compares a variation to a current experience allows you to ask questions about changes to your website or app and then collect data on the impact of that change.

Based on the assumptions in website optimization, the test allows to make analyzes about business processes that can be called "we know" instead of "we think". By measuring the impact of changes on your metrics, you can get an idea of the consequences of each change.

To sum it up, let's review once again what you need to do the A/B test.

- Basic data should be collected

- Goals must be set

- Establish an assumption (estimate how a particular element might perform better)

- A variable must be created (a variable must be created to oppose the existing element based on the assumption)

- Run the test (must take into account time, means, etc.) Be sure to test both versions (control and test) simultaneously to ensure validity of results

- Analyze the results – review the results to make sure the desired effect is achieved. Then choose which version performs better.